In [ ]:
data_2 = query_iwencai('2023年，地天板涨停')
data_2.to_excel('rw2.xlsx',sheet_name = 'Sheet1')


In [ ]:
import tushare as ts
import pandas as pd
import time

# 初始化Tushare
ts.set_token('2876ea85cb005fb5fa17c809a98174f2d5aae8b1f830110a5ead6211')
pro = ts.pro_api()

# 读取Excel文件
input_df = pd.read_excel(r'C:\Users\admin\Desktop\rw2.xlsx')

def safe_get_daily(ts_code):
    """获取日线数据"""
    for _ in range(3): 
        try:
            df = pro.daily(ts_code=ts_code, 
                         start_date='20230101', 
                         end_date='20231231',
                         fields='ts_code,trade_date,open,high,low,close,pre_close,vol')
            df['trade_date'] = pd.to_datetime(df['trade_date'])
            return df.sort_values('trade_date')
        except Exception as e:
            print(f"获取 {ts_code} 数据失败")
            time.sleep(1)
    return None

# 获取所有股票数据
all_data = []
for code in input_df['股票代码']:
    print(f"正在获取 {code} 数据...")
    df = safe_get_daily(code)
    if df is not None:
        all_data.append(df)
    time.sleep(0.2)  # 控制请求频率

# 合并保存数据
if all_data:
    full_df = pd.concat(all_data)
    output_path = r'C:\Users\admin\Desktop\2023年日线数据.xlsx'
    full_df.to_excel(output_path, index=False)
    print(f"数据已保存至 {output_path}")
    print(f"共包含 {len(full_df)} 条记录")
else:
    print("未能获取任何数据")

In [ ]:
data = pd.read_excel(r'C:\Users\admin\Desktop\2023年日线数据.xlsx')

In [ ]:
import pandas as pd

def find_ditianban_and_followup(data):

    # 确保日期是datetime类型并按日期排序
    data['date'] = pd.to_datetime(data['date'])
    data = data.sort_values(['code', 'date'])
    
    # 计算涨停价和跌停价
    data['limit_up'] = data['pre_close'] * 1.1
    data['limit_down'] = data['pre_close'] * 0.9
    
    
    ditianban_condition = (
        (data['low'] <= data['limit_down'] * 1.005) & 
        (data['high'] >= data['limit_up'] * 0.995) & 
        (data['close'] >= data['limit_up'] * 0.99)
    )
    
    # 找出所有地天板日期
    ditianban_days = data[ditianban_condition].copy()
    
    if ditianban_days.empty:
        return pd.DataFrame()  
    
    # 为每只股票计算后续涨跌幅
    results = []
    
    for _, row in ditianban_days.iterrows():
        stock_code = row['code']
        event_date = row['date']
        
        # 获取该股票的所有数据
        stock_data = data[data['code'] == stock_code].sort_values('date')
        
        # 找到事件日的位置
        event_idx = stock_data[stock_data['date'] == event_date].index[0]
        
        # 获取后续交易日数据
        next_day = stock_data.loc[event_idx + 1] if event_idx + 1 in stock_data.index else None
        day_after_3 = stock_data.loc[event_idx + 3] if event_idx + 3 in stock_data.index else None
        
        # 计算涨跌幅
        next_day_pct = (next_day['close'] / row['close'] - 1) * 100 if next_day is not None else None
        day3_pct = (day_after_3['close'] / row['close'] - 1) * 100 if day_after_3 is not None else None
        
        results.append({
            'code': stock_code,
            'name': row.get('name', ''),  
            'date': event_date,
            'close_price': row['close'],
            'next_day_pct': next_day_pct,
            'next_3day_pct': day3_pct,
            'volume': row['vol']
        })
    
    result_df = pd.DataFrame(results)
    
    return result_df

# 示例使用
if __name__ == "__main__":
   
    df = pd.DataFrame(data)
    
    result = find_ditianban_and_followup(df)
    
    # 导出到Excel
    if not result.empty:
        result.to_excel('ditianban_results.xlsx', index=False)
        print("地天板结果已保存到 ditianban_results.xlsx")
        print(result)
    else:
        print("没有找到地天板数据")